In [1]:
# install ngrok to emulate public IP / address
!wget -N https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip

for details.

--2022-01-07 11:38:01--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 54.161.241.46, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  38.3MB/s    in 0.3s    

2022-01-07 11:38:02 (38.3 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip


In [2]:
# TODO: sign up for an ngrok account
# then put your ngrok token below, uncomment, and execute
!./ngrok authtoken 23Kbw3CcDDS20BAb1UEm7UknyJn_5gwinxnqscxNH9qgbFdHx

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [3]:
# install a ngrok python package and a version of flask that works with it in 
# colab
!pip -q install flask-ngrok
!pip -q install flask==0.12.2
# !pip -q install flask_restful

# Run the app

In [4]:
# adding our python module to the cluster
%cd -q /home/dataproc
!ls inverted_index_gcp.py
from inverted_index_gcp import *

# uncomment the code below and execute to reload the module when you make 
# changes to search_frontend.py (after you upload again).
# import importlib
# importlib.reload(se)

inverted_index_gcp.py


In [5]:
import pyspark
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
from google.cloud import storage
from collections import defaultdict
from contextlib import closing
import pandas as pd
import numpy as np
import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

from builtins import *

bucket_name = 'fadlonbucket'

import pickle
def open_gcp(file_name):
    client = storage.Client(file_name)
    bucket = client.bucket(bucket_name)
    blob = bucket.get_blob('postings_gcp/' + file_name)
    return blob.open('rb')

def read_pickle(file_name):
    stream = open_gcp(file_name+".pkl")
    pick = pickle.load(stream)
    stream.close()
    print(file_name)
    return pick

def get_title_by_doc_id(doc_id):
    return doc_id_to_title_dic[doc_id]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
print("Start loading...")
inverted_title = read_pickle("title")
inverted_anchor = read_pickle("anchor")
page_rank_dict = read_pickle("page_rank_dict")
page_view = read_pickle("page_view")

Start loading...
title
anchor
page_rank_dict
page_view


In [7]:
inverted_body = read_pickle("text")
doc_id_to_title_dic = read_pickle("doc_id_to_title_dict")
print("Done!")

text
doc_id_to_title_dict
Done!


In [8]:
TUPLE_SIZE = 6       
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer
from contextlib import closing

def read_posting_list(inverted, w):
  with closing(MultiFileReader(bucket_name)) as reader:
    print(w)
    locs = inverted.posting_locs[w]
    print(locs)
    b = reader.read(locs, inverted.df[w] * TUPLE_SIZE)
    posting_list = []
    for i in range(inverted.df[w]):
      doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
      tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
      posting_list.append((doc_id, tf))
    return posting_list

from builtins import *
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage
import functools
import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb']  # TODO: calculate the corups stop words
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
stemmer = PorterStemmer()
all_stopwords = english_stopwords.union(corpus_stopwords)

def tokenize(text, stem=False):
  """
    This function aims in tokenize a text into a list of tokens.
    Moreover:
    * filter stopwords.
    * change all to lowwer case.
    * use stemmer
    
    Parameters:
    -----------
    text: string , represting the text to tokenize.    
    
    Returns:
    -----------
    list of tokens (e.g., list of tokens).
    """
  clean_text = []

  text = text.lower()
  tokens = [token.group() for token in RE_WORD.finditer(text)]
  for token in tokens:
    if token not in all_stopwords:
      if stem:
        token = stemmer.stem(token)
      clean_text.append(token)
  return clean_text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
def search_procedure(query):
    print("not implemented")
    return []

In [10]:
def search_body_procedure(query):
    print("not implemented")
    return []

In [11]:
def search_title_procedure(query):
    query = tokenize(query)
    results = []
    for term in query:
        try:
            results.append(read_posting_list(inverted_title, term))
        except:
            print("Term not in inverted_title: " + term)
            pass
    
    if len(results) != 0:
        results = functools.reduce(lambda a, b: a+b, results)
        results = map(lambda x: x[0], results)
        counter = Counter()
        counter.update(results)
        results = map(lambda x: (x[0], get_title_by_doc_id(x[0])), counter.most_common())

    return list(results)

In [12]:
def search_anchor_procedure(query):
    query = tokenize(query)
    results = []
    for term in query:
        try:
            results.append(read_posting_list(inverted_anchor, term))
        except:
            print("Term not in inverted_anchor: " + term)
            pass
        
    if len(results) != 0:
        results = functools.reduce(lambda a, b: a+b, results)
        results = map(lambda x: x[0], results)
        counter = Counter()
        counter.update(results)
        results = map(lambda x: (x[0], get_title_by_doc_id(x[0])), counter.most_common())
    return list(results)

In [13]:
def page_rank_procedure(wiki_ids):
    res = []
    for doc_id in wiki_ids:
        try:
            res.extend(page_rank_dict[doc_id])
        except:
            print("doc_id not in page_rank_dict: " + str(doc_id))
            res.append(0)
    return res

In [14]:
def page_view_procedure(lst):
    res = []
    for doc_id in lst:
        try:
            res.append(page_view[doc_id])
        except:
            print("doc_id not in page_view: " + str(doc_id))
            res.append(0)
    return res

In [15]:
from flask import Flask, request, jsonify

class MyFlaskApp(Flask):
    def run(self, host=None, port=None, debug=None, **options):
        super(MyFlaskApp, self).run(host=host, port=port, debug=debug, **options)

app = MyFlaskApp(__name__)
app.config['JSONIFY_PRETTYPRINT_REGULAR'] = False


@app.route("/search")
def search():
    ''' Returns up to a 100 of your best search results for the query. This is 
        the place to put forward your best search engine, and you are free to
        implement the retrieval whoever you'd like within the bound of the 
        project requirements (efficiency, quality, etc.). That means it is up to
        you to decide on whether to use stemming, remove stopwords, use 
        PageRank, query expansion, etc.

        To issue a query navigate to a URL like:
         http://YOUR_SERVER_DOMAIN/search?query=hello+world
        where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of up to 100 search results, ordered from best to worst where each 
        element is a tuple (wiki_id, title).
    '''
    res = []
    query = request.args.get('query', '')
    if len(query) == 0:
      return jsonify(res)
    # BEGIN SOLUTION
    res = search_procedure(query)
    # END SOLUTION
    return jsonify(res)

@app.route("/search_body")
def search_body():
    ''' Returns up to a 100 search results for the query using TFIDF AND COSINE
        SIMILARITY OF THE BODY OF ARTICLES ONLY. DO NOT use stemming. DO USE the 
        staff-provided tokenizer from Assignment 3 (GCP part) to do the 
        tokenization and remove stopwords. 

        To issue a query navigate to a URL like:
         http://YOUR_SERVER_DOMAIN/search_body?query=hello+world
        where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of up to 100 search results, ordered from best to worst where each 
        element is a tuple (wiki_id, title).
    '''
    res = []
    query = request.args.get('query', '')
    if len(query) == 0:
      return jsonify(res)
    # BEGIN SOLUTION
    res = search_body_procedure(query)
    # END SOLUTION
    return jsonify(res)

@app.route("/search_title")
def search_title():
    ''' Returns ALL (not just top 100) search results that contain A QUERY WORD 
        IN THE TITLE of articles, ordered in descending order of the NUMBER OF 
        QUERY WORDS that appear in the title. For example, a document with a 
        title that matches two of the query words will be ranked before a 
        document with a title that matches only one query term. 

        Test this by navigating to the a URL like:
         http://YOUR_SERVER_DOMAIN/search_title?query=hello+world
        where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of ALL (not just top 100) search results, ordered from best to 
        worst where each element is a tuple (wiki_id, title).
    '''
    res = []
    query = request.args.get('query', '')
    if len(query) == 0:
      return jsonify(res)
    # BEGIN SOLUTION
    res = search_title_procedure(query)
    # END SOLUTION
    return jsonify(res)

@app.route("/search_anchor")
def search_anchor():
    ''' Returns ALL (not just top 100) search results that contain A QUERY WORD 
        IN THE ANCHOR TEXT of articles, ordered in descending order of the 
        NUMBER OF QUERY WORDS that appear in anchor text linking to the page. 
        For example, a document with a anchor text that matches two of the 
        query words will be ranked before a document with anchor text that 
        matches only one query term. 

        Test this by navigating to the a URL like:
         http://YOUR_SERVER_DOMAIN/search_anchor?query=hello+world
        where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of ALL (not just top 100) search results, ordered from best to 
        worst where each element is a tuple (wiki_id, title).
    '''
    res = []
    query = request.args.get('query', '')
    if len(query) == 0:
      return jsonify(res)
    # BEGIN SOLUTION
    res = search_anchor_procedure(query)
    # END SOLUTION
    return jsonify(res)

@app.route("/get_pagerank", methods=['POST'])
def get_pagerank():
    ''' Returns PageRank values for a list of provided wiki article IDs. 

        Test this by issuing a POST request to a URL like:
          http://YOUR_SERVER_DOMAIN/get_pagerank
        with a json payload of the list of article ids. In python do:
          import requests
          requests.post('http://YOUR_SERVER_DOMAIN/get_pagerank', json=[1,5,8])
        As before YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of floats:
          list of PageRank scores that correrspond to the provided article IDs.
    '''
    res = []
#     wiki_ids = request.get_json()
    wiki_ids = request.get_json(force=True, silent=True, cache=False)
    if len(wiki_ids) == 0:
      return jsonify(res)
    # BEGIN SOLUTION
    res =  page_rank_procedure(wiki_ids)
    # END SOLUTION
    return jsonify(res)

@app.route("/get_pageview", methods=['POST'])
def get_pageview():
    ''' Returns the number of page views that each of the provide wiki articles
        had in August 2021.

        Test this by issuing a POST request to a URL like:
          http://YOUR_SERVER_DOMAIN/get_pageview
        with a json payload of the list of article ids. In python do:
          import requests
          requests.post('http://YOUR_SERVER_DOMAIN/get_pageview', json=[1,5,8])
        As before YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of ints:
          list of page view numbers from August 2021 that correrspond to the 
          provided list article IDs.
    '''
    res = []
#     wiki_ids = request.get_json()
    wiki_ids = request.get_json(force=True, silent=True, cache=False)
    if len(wiki_ids) == 0:
      return jsonify(res)
    # BEGIN SOLUTION
    res = page_view_procedure(wiki_ids)
    # END SOLUTION
    return jsonify(res)

# print(__name__)
# if __name__ == '__main__':
#     # run the Flask RESTful API, make the server publicly available (host='0.0.0.0') on port 8080
#     app.run(host='0.0.0.0', port=8080, debug=True)

In [16]:
from flask_ngrok import run_with_ngrok
from multiprocessing import Process

run_with_ngrok(app) 
server = Process(target=app.run)
server.start()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://eb24-34-67-3-75.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
doc_id not in page_rank_dict: 1
doc_id not in page_rank_dict: 5
doc_id not in page_rank_dict: 8


127.0.0.1 - - [07/Jan/2022 11:40:00] "POST /get_pagerank HTTP/1.1" 200 -
127.0.0.1 - - [07/Jan/2022 11:40:34] "POST /get_pagerank HTTP/1.1" 200 -
127.0.0.1 - - [07/Jan/2022 11:41:25] "POST /get_pagerank HTTP/1.1" 200 -


doc_id not in page_rank_dict: 1
doc_id not in page_rank_dict: 5
doc_id not in page_rank_dict: 8


127.0.0.1 - - [07/Jan/2022 11:43:01] "POST /get_pagerank HTTP/1.1" 200 -


doc_id not in page_rank_dict: 607
doc_id not in page_rank_dict: -1


127.0.0.1 - - [07/Jan/2022 11:44:02] "POST /get_pagerank HTTP/1.1" 200 -
127.0.0.1 - - [07/Jan/2022 11:44:53] "POST /get_pageview HTTP/1.1" 200 -


In [19]:
# server.terminate()
# server.join()

# Testing your app

Once your app is running you can query it. You can simply do that by navigating to the URL that ngrok gave you above or through code in a different python session. For example, once the frontend app is running, you can navigate to:
http://YOUR_SERVER_DOMAIN/search?query=hello+world where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io, which is printed above in Colab or that is your external IP on GCP.

The code below shows how to issue a query from python. This is also how our testing code will issue queries to your search engine, so make sure to test your search engine this way after you deploy it to GCP and before submission. Command line instructions for deploying your search engine to GCP are available at `run_frontend_in_gcp.sh`. Note that we will not only issue training queries to your search engine, but also test queries, i.e. queries that you've never seen before.

In [19]:
import json

with open('queries_train.json', 'rt') as f:
  queries = json.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'queries_train.json'

In [ ]:
def average_precision(true_list, predicted_list, k=40):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    precisions = []
    for i,doc_id in enumerate(predicted_list):        
        if doc_id in true_set:
            prec = (len(precisions)+1) / (i+1)            
            precisions.append(prec)
    if len(precisions) == 0:
        return 0.0
    return round(sum(precisions)/len(precisions),3)

In [ ]:
import requests
from time import time
# url = 'http://35.232.59.3:8080'
# place the domain you got from ngrok or GCP IP below. 
url = 'http://XXXX-XX-XX-XX-XX.ngrok.io'

qs_res = []
for q, true_wids in queries.items():
  duration, ap = None, None
  t_start = time()
  try:
    res = requests.get(url + '/search', {'query': q}, timeout=35)
    duration = time() - t_start
    if res.status_code == 200:
      pred_wids, _ = zip(*res.json())
      ap = average_precision(true_wids, pred_wids)
  except:
    pass
  
  qs_res.append((q, duration, ap))